In [16]:
import numpy as np
import pandas as pd

## Predictions mean

In [9]:
TRIALS = 3

In [10]:
empathy = []
distress = []

for trial in range(TRIALS):
    path = f"./predictions_EMP_trial{trial+1}.tsv"
    data = pd.read_csv(path, sep="\t", names=['empathy', 'distress'])
    empathy.append(data['empathy'])
    distress.append(data['distress'])

empathy = np.mean(empathy, axis=0)
distress = np.mean(distress, axis=0)

mean_pred_test = pd.DataFrame({'empathy': empathy, 'distress': distress})
mean_pred_test.to_csv('./predictions_EMP_mean.tsv', sep="\t", index=False, header=False)


In [12]:
# predictions mean from 3 trials, model trained on wassa 2022 training data
mean_pred_test

,empathy,distress
0,4.648031,3.065021
1,5.477338,4.998946
2,5.480438,4.806422
3,4.488974,5.146952
4,5.202304,5.138179
...,...,...
95,4.198460,5.548204
96,5.067682,4.465834
97,3.830879,4.920369
98,4.885243,5.188070


## Add prompt to data

In [3]:
def add_gold_prompt(df):
    for idx, row in df.iterrows():
        _, text_prompt_emp, _ = generate_prompt(
                                    row['essay'],
                                    row['article_id'],
                                    row['gender'],
                                    row['education'],
                                    row['race'],
                                    row['age'],
                                    row['income'],
                                    row['empathy_level'],
                                    row['distress_level']
                                    )
        df.at[idx, "prompt_emo"] = text_prompt_emp
    return df

In [6]:
from utils import generate_prompt, get_emp_levels

In [13]:
test_df = pd.read_csv('./datasets/WASSA23_essay_level_test_preproc.tsv', sep='\t')
test_df = get_emp_levels(test_df, mean_pred_test['empathy'], mean_pred_test['distress'])
test_df = add_gold_prompt(test_df)

In [14]:
# generate prompt for wassa 2023 test set from predictions on previously model
test_df

,conversation_id,article_id,essay,speaker_id,gender,education,race,age,income,speaker_number,...,disgust_count,joy_count,hope_count,empathy_count,distress_count,empathy_level,distress_level,prompt_bio,prompt_emp,prompt_emo
0,1192,6,"hi my dear friend, how r you?. Are you use...",45,2,6,1,25,50000,2,...,0.000000,0.214286,0.066667,2.908646,2.725509,medium,medium,"An essay about the article 6, written by a 25 ...",The essay expresses medium empathy and medium ...,The essay expresses medium empathy and medium ...
1,1192,6,The loss of human life is always a tragic even...,65,1,6,1,34,30000,1,...,0.066667,0.044444,0.035714,3.130824,2.885318,high,medium,"An essay about the article 6, written by a 34 ...",The essay expresses medium empathy and medium ...,The essay expresses high empathy and medium di...
2,1059,6,"Hello mate, it's been a long time since I met ...",53,2,6,1,64,40000,2,...,0.000000,0.058824,0.055556,2.678742,2.543031,high,medium,"An essay about the article 6, written by a 64 ...",The essay expresses medium empathy and medium ...,The essay expresses high empathy and medium di...
3,1059,6,us America and the security council adopts res...,92,2,6,1,35,5000,1,...,0.048780,0.000000,0.000000,2.689422,2.592165,medium,high,"An essay about the article 6, written by a 35 ...",The essay expresses medium empathy and low dis...,The essay expresses medium empathy and high di...
4,1060,6,My thought was NATO and government forces are ...,47,2,7,1,34,75000,1,...,0.031250,0.000000,0.000000,2.619263,2.593569,high,high,"An essay about the article 6, written by a 34 ...",The essay expresses medium empathy and medium ...,The essay expresses high empathy and high dist...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1183,19,Check out this story. It is an outrage that so...,143,2,6,1,42,50000,1,...,0.066667,0.100000,0.000000,2.762659,2.734730,medium,high,"An essay about the article 19, written by a 42...",The essay expresses medium empathy and low dis...,The essay expresses medium empathy and high di...
96,1184,19,I just read a story about a person who was bro...,145,1,4,1,63,55000,2,...,0.041667,0.041667,0.000000,2.777718,2.782064,high,medium,"An essay about the article 19, written by a 63...",The essay expresses low empathy and medium dis...,The essay expresses high empathy and medium di...
97,1184,19,I just read this article and it's really messe...,144,2,6,3,31,36000,1,...,0.137931,0.034483,0.000000,2.646881,2.650662,medium,medium,"An essay about the article 19, written by a 31...",The essay expresses low empathy and medium dis...,The essay expresses medium empathy and medium ...
98,1190,19,I just read about a man who is being deported ...,130,1,2,1,30,36000,2,...,0.133333,0.000000,0.000000,2.758860,2.687240,medium,high,"An essay about the article 19, written by a 30...",The essay expresses low empathy and medium dis...,The essay expresses medium empathy and high di...


In [15]:
test_df.to_csv('./WASSA23_essay_level_test_gold_prompt.tsv', sep='\t', index=False)